In [ ]:
#Variable Definition
task_type= 'QA' #'QA','Data2txt','Summary','truthqa'
model_type= 'llama' #'llama','qwen'

#qwen's truthqa is not implemented

In [ ]:
if task_type == 'QA':
    dataset_name ='qa'
elif task_type == 'Data2txt':
    dataset_name = 'data2txt'
elif task_type == 'Summary':
    dataset_name = 'summary'

In [1]:
#import
import torch
import os

#GPU Configuration (Specify the GPU to Use)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

#Change Hugging Face Cache (If Necessary)
#os.environ['HF_HOME'] = "/home/code/hallucination/llm"

In [2]:
# Function to Display Available GPUs
def print_available_gpus():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

print_available_gpus()

Number of available GPUs: 3
GPU 0: NVIDIA RTX 6000 Ada Generation
GPU 1: NVIDIA RTX 6000 Ada Generation
GPU 2: NVIDIA RTX 6000 Ada Generation


In [3]:
import resource
import logging
resource.setrlimit(resource.RLIMIT_CORE, (0, 0))
logging.info("Coredump generation has been disabled.")

In [5]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from tqdm import tqdm


from gen_features import *

In [ ]:
import os
os.chdir('/home/code')

In [7]:
if (task_type == 'QA') or (task_type == 'Data2Txt') or (task_type == 'Summary'):
    df = pd.read_json('./data/ragtruth_datasets/rag_truth_merged_data.jsonl', lines=True)
    
    
    #Specify Task
    if task_type == 'QA':
        df=df[df['task_type'].apply(lambda x: x == 'QA')].reset_index(drop=True)
    elif task_type == 'Data2Txt':
        df=df[df['task_type'].apply(lambda x: x == 'Data2txt')].reset_index(drop=True)
    elif task_type == 'Summary':
        df=df[df['task_type'].apply(lambda x: x == 'Summary')].reset_index(drop=True)


elif task_type == 'truthqa':
    df = pd.read_csv('./data/truthfulqa_datasets/eval_examples.csv')

In [ ]:
display(df)

,source_id,model,temperature,labels,split,quality,response,task_type,source,source_info,prompt,combined_text
0,14292,gpt-4-0613,0.70,[],train,good,The phone numbers for several butcher shops ar...,QA,MARCO,"{'question': 'butcher shop phone number', 'pas...",Briefly answer the following question:\nbutche...,Briefly answer the following question:\nbutche...
1,14292,gpt-3.5-turbo-0613,0.70,[],train,good,Butcher Shop - Hayward phone number: (510) 889...,QA,MARCO,"{'question': 'butcher shop phone number', 'pas...",Briefly answer the following question:\nbutche...,Briefly answer the following question:\nbutche...
2,14292,mistral-7B-instruct,0.70,"[{'start': 102, 'end': 214, 'text': 'However, ...",train,good,"Based on the given passages, there are several...",QA,MARCO,"{'question': 'butcher shop phone number', 'pas...",Briefly answer the following question:\nbutche...,Briefly answer the following question:\nbutche...
3,14292,llama-2-7b-chat,0.85,[],train,good,"Based on the provided passages, the butcher sh...",QA,MARCO,"{'question': 'butcher shop phone number', 'pas...",Briefly answer the following question:\nbutche...,Briefly answer the following question:\nbutche...
4,14292,llama-2-13b-chat,0.70,[],train,good,"Based on the given passages, the butcher shop ...",QA,MARCO,"{'question': 'butcher shop phone number', 'pas...",Briefly answer the following question:\nbutche...,Briefly answer the following question:\nbutche...
...,...,...,...,...,...,...,...,...,...,...,...,...
5929,12460,gpt-3.5-turbo-0613,0.70,[],train,good,Eating tomatoes can provide nutrition benefits...,QA,MARCO,{'question': 'what are nutrition benefits eati...,Briefly answer the following question:\nwhat a...,Briefly answer the following question:\nwhat a...
5930,12460,mistral-7B-instruct,0.70,"[{'start': 193, 'end': 223, 'text': 'and suppo...",train,good,"Based on the given passages, it appears that e...",QA,MARCO,{'question': 'what are nutrition benefits eati...,Briefly answer the following question:\nwhat a...,Briefly answer the following question:\nwhat a...
5931,12460,llama-2-7b-chat,0.70,"[{'start': 634, 'end': 690, 'text': 'reducing ...",train,good,"Based on the provided passages, here are the n...",QA,MARCO,{'question': 'what are nutrition benefits eati...,Briefly answer the following question:\nwhat a...,Briefly answer the following question:\nwhat a...
5932,12460,llama-2-13b-chat,1.00,[],train,good,"Based on the provided passages, here are the n...",QA,MARCO,{'question': 'what are nutrition benefits eati...,Briefly answer the following question:\nwhat a...,Briefly answer the following question:\nwhat a...


In [ ]:
#Specify Model
if model_type == 'llama':
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    hf_token = "aaaaa" #Set Token Manually
    model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True,torch_dtype=torch.float16,token=hf_token)
    tokenizer = AutoTokenizer.from_pretrained(model_name,token=hf_token)
    

elif model_type == 'qwen':
    model_name = "Qwen/Qwen2.5-7B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(model_name, output_attentions=True,torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(model_name)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)

matrix_device_v = torch.device("cuda:1")
matrix_device_o = torch.device("cuda:2")

In [ ]:
#Define Prompt + Also Serve as a Test
if model_type == 'llama':
    if task_type != 'truthqa':
        system_prompt="You are an excellent system, generating output according to the instructions."
        header_prompt="<|start_header_id|>system<|end_header_id|>"
        user_prompt="<|eot_id|><|start_header_id|>user<|end_header_id|>"
        assistant_prompt="<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        
        
        #test
        message_prompt= df.loc[0, 'prompt']
        response = df.loc[0, 'response']
        full_prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n{response}<|eot_id|>"
        output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio=get_features_no_generate(full_prompt, model, tokenizer, device, matrix_device_v, matrix_device_o)
        
        
        
        

    else:
        system_prompt="You are an excellent question-answering system. Respond to the given questions in 30 words or less."
        header_prompt="<|start_header_id|>system<|end_header_id|>"
        user_prompt="<|eot_id|><|start_header_id|>user<|end_header_id|>"
        assistant_prompt="<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        
        #test
        message_prompt="Q: What is the capital of Japan?\nA:"
        temperature=1.50
        top_k=100
        top_p=1.5
        max_new_tokens=70
        seed=42
        
        prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n"
        output_text,generated_text, output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio=get_features_with_generate(prompt, model, tokenizer, device, matrix_device_v, matrix_device_o,temperature, top_k, top_p, seed, max_new_tokens)
        


elif model_type=='qwen':
    if task_type != 'truthqa':
        system_prompt="You are an excellent system, generating output according to the instructions."
        header_prompt="<|im_start|>system"
        user_prompt="<|im_end|>\n<|im_start|>user"
        assistant_prompt="<|im_end|>\n<|im_start|>assistant"
        
        #test
        message_prompt= df.loc[0, 'prompt']
        response = df.loc[0, 'response']
        full_prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n{response}<|im_end|>"
        
        output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio=get_features_no_generate(full_prompt, model, tokenizer, device, matrix_device_v, matrix_device_o)
    

print(output_tokens)
print(raw_key_avg[0])

In [ ]:
data = []

In [ ]:
#Execute (Including Saving)
if task_type != 'truthqa':
    for i in tqdm(range(len(df))):
        message_prompt= df.loc[i, 'prompt']
        response = df.loc[i, 'response']

        if model_type == 'llama':
            full_prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n{response}<|eot_id|>"
        elif model_type == 'qwen':
            full_prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n{response}<|im_end|>"
        
        output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio=get_features_no_generate(full_prompt, model, tokenizer, device, matrix_device_v, matrix_device_o)            
        data.append([full_prompt,output_tokens,raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio])
        
    result_df = pd.DataFrame(data, columns=['full_prompt','output_tokens','raw_key_avg', 'raw_query_entropy', 'raw_key_entropy','raw_lookback_ratio', 'norm_key_avg', 'norm_query_entropy', 'norm_key_entropy','norm_lookback_ratio'])

    #Merge result_df and df
    df = pd.concat([df, result_df], axis=1)
    
    #save
    df.to_pickle(f'./data/saves/{model_type}_{dataset_name}.pkl')
    

else: #In the Case of TruthfulQA
    temperature=1.50
    top_k=100
    top_p=1.5
    max_new_tokens=70
    seed_values=[2,12,22,32,42,52,62,72,82,92]
    
    
    for i in tqdm(range(len(df))):
        question = df.loc[i, 'Question']
        message_prompt=f"Q: {question}\nA:"
        prompt=f"{header_prompt}\n{system_prompt}{user_prompt}\n{message_prompt}{assistant_prompt}\n"
        
        for seed in seed_values:
            output_text,generated_text, output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio=get_features_with_generate(prompt, model, tokenizer, device, matrix_device_v, matrix_device_o,temperature, top_k, top_p, seed, max_new_tokens)
            data.append([question,prompt,output_text,generated_text, output_tokens, raw_key_avg, raw_query_entropy, raw_key_entropy,raw_lookback_ratio,norm_key_avg, norm_query_entropy, norm_key_entropy,norm_lookback_ratio])
    
    
    # Convert List to DataFrame
    result_df = pd.DataFrame(data, columns=['question', 'prompt', 'output_text', 'generated_text', 'output_tokens', 'raw_key_avg', 'raw_query_entropy', 'raw_key_entropy', 'raw_lookback_ratio', 'norm_key_avg', 'norm_query_entropy', 'norm_key_entropy', 'norm_lookback_ratio'])
    
    #save
    result_df.to_pickle(f'./data/saves/{model_type}_{task_type}.pkl')